# Retrieval-Augmented Generation (RAG)

In [3]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [14]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")

In [5]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

/home/vscode/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 10.4MiB/s]


[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [16]:
results_qna = nutrition_qna.query(query_texts=["how long is the pregnancy?"], n_results=2)
for i, doc in enumerate(results_qna["documents"][0]):
    print(sorted(results_qna["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('is_pregnancy', True)]
Question: Which hormone reaches its peak at around 8 weeks gestation and what does it do?
        Answer: Human choironic gonadotropin (HCG) is the hormone that peaks at approximately 8 weeks of gestation, maintaining the corpus lutem during pregnancy.

        This Q&A pair provides information about nutrition and health topics.


[('is_pregnancy', True)]
Question: What physical changes can one anticipate during the middle three months of pregnancy?
        Answer: In weeks 13-27, a woman's weight will increase along with her appetite. She may also face back pain and other discomforts like leg cramps and heartburn.

        This Q&A pair provides information about nutrition and health topics.




In [11]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [34]:
results_qna

{'ids': [['qa_139', 'qa_2046']],
 'embeddings': None,
 'documents': [['Question: Which hormone reaches its peak at around 8 weeks gestation and what does it do?\n        Answer: Human choironic gonadotropin (HCG) is the hormone that peaks at approximately 8 weeks of gestation, maintaining the corpus lutem during pregnancy.\n\n        This Q&A pair provides information about nutrition and health topics.',
   "Question: What physical changes can one anticipate during the middle three months of pregnancy?\n        Answer: In weeks 13-27, a woman's weight will increase along with her appetite. She may also face back pain and other discomforts like leg cramps and heartburn.\n\n        This Q&A pair provides information about nutrition and health topics."]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'is_pregnancy': True}, {'is_pregnancy': True}]],
 'distances': [[1.051164150238037, 1.1072723865509033]]}

In [39]:
@function_tool
def qna_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up nutrition and pregnancy information for specific questions. If the answer relates to pregnancy, the metadata "is_pregnancy" is True, otherwise the question and answer pair relates to nutrition

    Args:
        query: The information to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing a question and answer pair.
    """

    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        question_answer_pair = results["documents"][0][i]
        metadata = results["metadatas"][0][i]
        is_pregnancy_question = metadata["is_pregnancy"]

        formatted_results.append(
            f"{question_answer_pair}: is pregunancy question? {is_pregnancy_question}"
        )

    return "Q and A pair :\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [10]:
calorie_lookup_tool('spread cheese')

'Nutrition Information:\nCheese Spread (Spreads): 290.0 calories per 100g\nCheese Spread (Cheese): 290.0 calories per 100g\nPhiladelphia Cream Cheese (Spreads): 342.0 calories per 100g'

In [38]:
qna_lookup_tool('pregnancy weeks')

"Q and A pair :\nQuestion: What physical changes can one anticipate during the middle three months of pregnancy?\n        Answer: In weeks 13-27, a woman's weight will increase along with her appetite. She may also face back pain and other discomforts like leg cramps and heartburn.\n\n        This Q&A pair provides information about nutrition and health topics.: is pregunancy question? True\nQuestion: What kind of physical changes might a woman notice in her body during weeks 13-27 of pregnancy?\n        Answer: She may start to gain weight more rapidly and experience back pain as the baby grows.\n\n        This Q&A pair provides information about nutrition and health topics.: is pregunancy question? True\nQuestion: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?\n        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartbu

In [ ]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    """,
    tools=[calorie_lookup_tool]
)

In [40]:
nutrition_pregnancy_qa_agent = Agent(
    name="Nutrition and Pregnancy Assistant",
    instructions="""
    You are a helpful assistant giving out nutrition, calorie and pregnancy information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool. If you need information regarding pregnancy or nutrition use the qna_lookup_tool
    """,
    tools=[calorie_lookup_tool,qna_lookup_tool]
)




In [13]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)

- Calories per 100 g: Banana 89 kcal; Apple 52 kcal.
- Typical single fruit estimates:
  - Medium banana (~118 g): ~105 kcal
  - Medium apple (~182 g): ~95 kcal
- Total for one banana + one apple (typical sizes): ~200 kcal

If you have exact weights, I can compute a precise total.


In [42]:
with trace("Nutrition and Pregnancy Assistant"):
    result = await Runner.run(
        nutrition_pregnancy_qa_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)

- Total calories (one medium banana + one medium apple): about 200 kcal (banana ~105 kcal; apple ~95 kcal).
- Calories per 100 g: banana 89 kcal; apple 52 kcal.


In [44]:
with trace("Nutrition and Pregnancy Assistant"):
    result = await Runner.run(
        nutrition_pregnancy_qa_agent,
        "what are the biggest physical changes in pregnancy?",
    )
    print(result.final_output)

Common big physical changes in pregnancy:

- Uterus expands dramatically, causing abdominal growth and a noticeable belly
- Weight gain (varies by person and health guidance)
- Breasts enlarge, feel tender, and nipples may darken
- Posture and back/pelvic pain as the belly grows
- Increased blood volume and heart rate; fatigue is common
- Skin changes: stretch marks, linea nigra, and sometimes rashes or darkening of areas
- Swelling (edema) in feet/ankles later in pregnancy
- Gastrointestinal changes: heartburn, constipation
- Fetal movements develop and become noticeable later

Note: experiences vary; discuss any concerning or severe symptoms with a healthcare provider.
